# Chapter 1: Initial Query with ChatGPT

Dear user, you are part of a consulting firm tasked to: \
[1] identify opportunities for design improvements for a specific product \
[2] conduct benchmarking with competitors whenever appropriate.

In this initial stage, we will initialize the **product** \
and gather initial insights and directions via querying a LLM (**ChatGPT**).

### TO DO SECTION

In [12]:
'''
Dear user, enter your Product here!
'''

product = "PICO 4 All-in-One VR Headset"

### RUN AS INTENDED (DO NOT CHANGE ANYTHING.)

##### Set Up

In [13]:
! pip install python-dotenv

In [14]:
""" Set up OpenAI API key """
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")

In [15]:
""" Create directory """
import shutil

if os.path.exists('support'):                     # Delete 'support' folder if it exists
    shutil.rmtree('support')

search_terms = product

try:                                              # Create directory named after search terms
    os.makedirs("support/%s" % search_terms)
    print("Directory", search_terms, "created")
except FileExistsError:
    print("Directory", search_terms, "exists")

Directory PICO 4 All-in-One VR Headset created


In [16]:
try:                                              # Create directory to store ChatGPT
    os.makedirs("support/%s/%s" % (search_terms, "ChatGPT"))
    print("Directory ChatGPT created")
except FileExistsError:
    print("Directory ChatGPT exists")

Directory ChatGPT created


##### DESIGN OPPORTUNITIES QUERY with ChatGPT

In [17]:
""" Design Opportunities Query with ChatGPT """
import pickle
import pandas as pd
import json

from openai import OpenAI
client = OpenAI()

question =  f"Identify and rank ten opportunities for design improvements with the {product}."
template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised explanation with a maximum of 45 characters. If you don't know the answer to the question, strictly state 'I don't know'."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        # adjust persona and temperature
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"), "wb"))

data_string = pd.read_pickle("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)

components = list(data.keys())
descriptions = list(data.values())

df = pd.DataFrame({'Component': components, 'Description': descriptions})

print("Design Opportunities Query with ChatGPT:")
print(df)


Design Opportunities Query with ChatGPT:
            Component                                      Description
0            Headband                Improve comfort and adjustability
1         Controllers             Enhance ergonomics and button layout
2            Speakers         Improve sound quality and volume control
3             Display          Increase resolution for clearer visuals
4       Charging Port             Introduce fast charging capabilities
5         Ventilation  Improve heat dissipation to prevent overheating
6              Weight         Reduce overall weight for better comfort
7  Software Interface           Improve user experience and navigation
8      Motion Sensors             Enhance tracking precision and speed
9        Battery Life               Increase duration for longer usage


##### BENCHMARKING QUERY with ChatGPT

In [18]:
""" BENCHMARKING QUERY with ChatGPT """

""" Get 3 competitors """
import pickle
import pandas as pd
import json

search_terms = product

from openai import OpenAI
client = OpenAI()

question =  f"Identify three competitors of the {product} and the top ten physical components of each competitor."
template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as the product name (do not add numbering) and the value as a Python list of the ten components. If you don't know the answer to the question, strictly state ' '."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        # adjust persona and temperature
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"), "wb"))

""" Benchmarking """
print("Benchmarking Query with ChatGPT:")
data_string = pd.read_pickle("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)

competitors = list(data.keys())
components = list(data.values())
client = OpenAI()

for i in range(len(competitors)):
    question = f"Using the {product}, conduct benchmarking with {competitors[i]} with regards to these ten physical components: {data[competitors[i]]}."
    template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised explanation with a maximum of 45 characters. If you don't know the answer to the question, strictly state 'I don't know.'."
    prompt = question + template

    chat_completion = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,  # adjust persona and temperature
    )

    result = chat_completion.choices[0].message.content

    pickle.dump(result, open("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]), "wb"))

    data_string = pd.read_pickle("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]))
    benchmarking_data = json.loads(data_string)

    components = list(benchmarking_data.keys())
    descriptions = list(benchmarking_data.values())

    df = pd.DataFrame({'Component': components, 'Benchmarking': descriptions})

    print(competitors[i])
    print(df)
    print("\n")


Benchmarking Query with ChatGPT:
Oculus Quest 2
               Component                                    Benchmarking
0             VR Headset  Pico is lighter, Quest 2 has better resolution
1  Two Touch Controllers              Both have similar ergonomic design
2         Charging Cable                     Both use USB-C for charging
3          Power Adapter                Both have similar power adapters
4         Glasses Spacer                  Quest 2 includes, Pico doesn't
5             Head Strap              Quest 2 is adjustable, Pico is not
6            USB-C Cable             Both headsets include a USB-C cable
7           AA Batteries              Required for Quest 2, not for Pico
8  Frame Friendly Spacer                 Included with Quest 2, not Pico
9     Instruction Manual          Both have detailed instruction manuals


HTC Vive Pro
           Component                                       Benchmarking
0         VR Headset  PICO is lighter but Vive Pro has better 